In [1]:
import IPython
import simpy
import concurrent
import myutils

In [2]:
G = myutils.load_map()

In [ ]:
class StationModel():
    stations = set()
    def __init__(self, routeId, varId) -> None:
        src = myutils.STOPS_FILE + str(varId)
        api = myutils.STOPS_API(id=routeId, varId=varId)
        stops_df = myutils.load_data(route=routeId, source=src, api=api)
        self.stops_df = stops_df[["StopId", "Lat", "Lng"]]
        self.stops_df = self.stops_df.rename(columns={"StopId":"stop_id", "Lat":"lat", "Lng":"lng"})

        StationModel.stations.update(self.stops_df["stop_id"].tolist())

In [ ]:
class BusModel():
    def __init__(self, id) -> None:
        self.stations = dict()
        self.paths_df = dict()
        timetables_df = myutils.load_data(route=id, source=myutils.TIMETABLE_FILE, api=myutils.TIMETABLE_API(id))
        for varId in timetables_df["RouteVarId"]:
            self.stations[varId] = StationModel(routeId=id, varId=varId)
            self.paths_df[varId] = myutils.load_data(route=id, source=myutils.PATHS_FILE, api=myutils.PATHS_API(id, varId=varId))
        
    def prepare_path(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            params = [(varId, self.paths_df[varId], self.stations[varId].stops_df, G) for varId in self.paths_df.keys()]
            results = executor.map(myutils.get_routes_from_paths, params)
            for result in results:
                id, df = result
                self.paths_df[id] = df

In [ ]:
buses = list()
bus_ids = [8, 51, 30]
for id in bus_ids:
    bus = BusModel(id)
    buses.append(bus)
    bus.prepare_path()

In [ ]:
myutils.graph_all_routes(buses, G)

TypeError: graph_all_routes() takes 1 positional argument but 2 were given

In [ ]:
def get_travel_times(df):
    return df["route"].apply(lambda route: sum(ox.utils_graph.get_route_edge_attributes(G, route, "travel_time")))

2439.7